In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'covid19-global-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1062019%2F3640943%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T214146Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D21bb1a02bbd6e83e1b528a23441838556f3a3a39267b3c68cfa61e4d7f388358d56169e94ad819e66ae08832e7e2cf1399a24878935ee278cb0db43b12f99d5cb4dd38748cde803d255db80ae5c90140937c417794edeee12831b514bb337392ca5c4c26896134702c5fcd65d87e868592edd9c7b8cb6c71e3fc4c4dd1b8793082525b3f68bf71c386b08b53279295ebf930795457f8adde9ba9a77731f05eb5535e8c274be544cca162d312445aea9cbc8636fbecfa2b740b1825d2d26bd42332c73324383917acfc7b47bc6697eeb649d34012a0fd20429a7ae54ae7d7d8bd0058eb419dd97dc97cc427a2b09671ecbd5eeedcb080980ddf74d1a374ffccc9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Covid-19 Data: Analytics, Visualizations, & Modeling

In 2020, the covid-19 pandemic rapidly spread through the world promptly sent the world into a state of panic and a series of lockdowns in hopes of preventing the virus from spreading further. Ultimately, between a combination of government policies, social changes,a vaccine and an overall increase in sanitation, by May 2023, the covid-19 pandemic was officially declared over.

## The Data

<i>"This data was scraped from woldometers.info on 2022-05-14 by Joseph Assaker. 225 countries are represented in this data."</i>

<i>"All of countries have records dating from 2020-2-15 until 2022-05-14 (820 days per country).
That's with the exception of China, which has records dating from 2020-1-22 until 2022-05-14 (844 days per country), and Palau which has records dating from 2021-8-25 until 2022-05-14 (263 days per country)."</i>

--------------------------------------------------------------------------------------------------------
**Summary Data Columns Description:**
    
- **country:** designates the Country in which the the row's data was observed.
    
- **continent:** designates the Continent of the observed country.
    
- **total_confirmed:** designates the total number of confirmed cases in the observed country.
    
- **total_deaths:** designates the total number of confirmed deaths in the observed country.
    
- **total_recovered:** designates the total number of confirmed recoveries in the observed country.
    
- **active_cases:** designates the number of active cases in the observed country.
    
- **serious_or_critical:** designates the estimated number of cases in serious or critical conditions in the observed country.

- **total_cases_per_1m_population:** designates the number of total cases per 1 million population in the observed country.
    
- **total_deaths_per_1m_population:** designates the number of total deaths per 1 million population in the observed country.
    
- **total_tests:** designates the number of total tests done in the observed country.
    
- **total_tests_per_1m_population:** designates the number of total test done per 1 million population in the observed country.
    
- **population:** designates the population count in the observed country.
    
**Daily Data Columns Description:**
    
- **date:** designates the date of observation of the row's data in YYYY-MM-DD format.
    
- **country:** designates the Country in which the the row's data was observed.
    
- **cumulative_total_cases:** designates the cumulative number of confirmed cases as of the row's date, for the row's country.
    
- **daily_new_cases:** designates the daily new number of confirmed cases on the row's date, for the row's country.
    
- **active_cases:** designates the number of active cases (i.e., confirmed cases that still didn't recover nor die) on the row's date, for the row's country.
    
- **cumulative_total_deaths:** designates the cumulative number of confirmed deaths as of the row's date, for the row's country.
    
- **daily_new_deaths:** designates the daily new number of confirmed deaths on the row's date, for the row's country.

## Libraries


In [ ]:
#Libraries for Statistical Analysis
import numpy as np #Numerical Manipulation
import pandas as pd #Dataframe Creation, Feature Engineering
import math #Additional Numerical Manipulation
from sklearn.linear_model import LinearRegression #Linear Regression


#Plotting and Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Reading in the "activity" csv file into a pandas dataframe
data = pd.read_csv('/kaggle/input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv')
data_summary = pd.read_csv('/kaggle/input/covid19-global-dataset/worldometer_coronavirus_summary_data.csv')

## (1) Statistical Analysis

Exploring various stastical methods I can perform on the datasets

### (1.1) Summary Statistics

General summary statistics of both datasets, as provided by pandas functions

In [ ]:
data.describe()

In [ ]:
data_summary.describe()

In [ ]:
data.drop(['date','country'],axis=1).corr()

In [ ]:
data_summary.drop(['country','continent'],axis=1).corr()

### (1.2) Additional Summary Metrics

Additional metrics that are not shown by general summary stats from pandas

In [ ]:
#Number of patients that had covid and survived
ad_metrics = pd.DataFrame()

ad_metrics["country"] = data_summary["country"]
ad_metrics["non_critical_cases"] = data_summary['total_confirmed'] - data_summary['serious_or_critical']
ad_metrics["test_confirmed_ratio"] = data_summary["total_tests"] / data_summary['total_confirmed']
ad_metrics['survival_ratio'] = data_summary['total_recovered'] / data_summary['total_deaths']

ad_metrics = ad_metrics.set_index('country')
ad_metrics = ad_metrics.replace({np.nan:0})

ad_metrics

## (2) Feature Engineering

To compare the impact of the United States vs. the rest of the world, I am creating two pandas dataframes named us_data and global_data, so that I can easily model U.S stats to the global stats

In [ ]:
#Creating the global dataframe
mask = data['country'] != 'USA'
global_data = data[mask]

#Creating the us dataframe
mask = data['country'] == 'USA'
us_data = data[mask]

#Reindexing for the US dataset
index = np.arange(0,len(us_data),1)
us_data = us_data.set_index(index)

#Reindexing for the global dataset
index = np.arange(0,len(global_data),1)
global_data = global_data.set_index(index)

global_data['active_cases'] = global_data['active_cases'].fillna(0)

## (3) Visualizations

### (3.1) Visualizing US Covid-19 [2/15/2020 - 5/14/2022]

In [ ]:
#Using linear regression to create a trend line of active cases
X = np.arange(0,len(us_data),1).reshape(-1,1)
y = us_data['active_cases']
model = LinearRegression().fit(X,y)
y_pred = model.predict(X)

#Creating the plot
plt.figure(figsize=(8,5))
#plt.plot(us_data['cumulative_total_cases'],label="Cumulative Total Cases")
#plt.plot(us_data['daily_new_cases'],label="Daily New Cases")
plt.plot(us_data['active_cases'],label="Active Cases", color='steelblue')
plt.plot(y_pred,label="Active Cases (Trendline)",color='steelblue',linestyle='dashed')
plt.plot(us_data['cumulative_total_deaths'],label="Deaths", color='black')
plt.xlabel('Days')
plt.ylabel('Cases (in millions)')
plt.title("US Covid Data [2/15/2020 - 5/14/2022]")
plt.axvline(x=300,color='r',label='Vaccine Released',linestyle='dashed')
plt.axvline(x=583,color='y',label='Booster #1',linestyle='dashed')
plt.axvline(x=765,color='g',label='Booster #2',linestyle='dashed')
plt.legend()
plt.show()

### (3.2) Heatmap of top 10 countries w/ the highest populations [Scaled by 1e^6]

In [ ]:
#mask = data_summary['country'] != "USA"
#heatmap = data_summary[mask]
heatmap = data_summary
heatmap = heatmap.set_index('country')
heatmap = heatmap.sort_values('population',ascending=False)[0:9]
heatmap = heatmap.drop(['continent','population','total_tests'],axis=1)
heatmap = heatmap * .000001
plt.xlabel('Category')
plt.ylabel('Country')
plt.title('Global Heatmap [Scaled by 1e^6, descending by pop.]')
sns.heatmap(heatmap, cmap='YlGnBu',annot=True,linewidths=2)